In [5]:
# analyze_accidents_updated.py
"""
Analyze US Accidents CSV to identify patterns related to:
- road conditions
- weather
- time of day

Outputs:
 - PNG charts in ./outputs/
 - CSV summaries in ./outputs/
 - HTML maps: accident_heatmap.html and top_clusters.html in ./outputs/
"""

import os
import math
import random
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap

# --------- CONFIG ----------
DATA_PATH = r"M:\Internship task\Task 5\US_Accidents_March23.csv"
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# columns we expect (common names in the Kaggle dataset)
LAT_COL = "Start_Lat"
LNG_COL = "Start_Lng"
TIME_COL = "Start_Time"
WEATHER_COL = "Weather_Condition"
SEV_COL = "Severity"

# candidate road condition boolean fields (if present)
ROAD_COLS = ["Traffic_Signal", "Crossing", "Junction", "Station", "Stop", "Turning_Loop",
             "Amenity", "Bump", "Give_Way", "No_Exit", "Railway", "Roundabout", "Traffic_Calming"]

# ---------- HELPERS ----------
def try_load_with_dtypes(path):
    """Attempt to load full CSV with memory-optimized dtypes."""
    dtype_map = {
        "Severity": "int8",
        "State": "category",
        "Zipcode": "category",
        "Weather_Condition": "category",
        "Country": "category",
        "Timezone": "category",
        "Side": "category"
    }
    use_parse_dates = [TIME_COL]
    print("Attempting optimized full load...")
    df = pd.read_csv(path, dtype=dtype_map, parse_dates=use_parse_dates, low_memory=True)
    return df

def load_chunk_aggregations(path, chunk_size=250_000):
    """
    Read CSV in chunks to compute aggregated summaries without loading full file.
    Returns aggregated DataFrames and a sampled dataframe for mapping.
    """
    print("Falling back to chunked processing (safe for low memory).")
    chunker = pd.read_csv(path, parse_dates=[TIME_COL], chunksize=chunk_size, low_memory=True)

    # accumulators
    hour_series = pd.Series(0, index=range(24), dtype="int64")
    weekday_series = pd.Series(0, index=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], dtype="int64")
    weather_counts = {}
    road_counts = {col: 0 for col in ROAD_COLS}
    severity_weather = {}
    sample_rows = []

    for i, chunk in enumerate(chunker):
        # basic cleaning
        chunk = chunk.dropna(subset=[TIME_COL])
        chunk[TIME_COL] = pd.to_datetime(chunk[TIME_COL], errors="coerce")
        chunk = chunk.dropna(subset=[TIME_COL])
        chunk["hour"] = chunk[TIME_COL].dt.hour
        chunk["weekday"] = chunk[TIME_COL].dt.day_name()

        # hours
        hour_series = hour_series.add(chunk["hour"].value_counts().reindex(range(24), fill_value=0), fill_value=0)
        # weekdays
        weekday_series = weekday_series.add(chunk["weekday"].value_counts().reindex(weekday_series.index, fill_value=0), fill_value=0)

        # weather counts
        if WEATHER_COL in chunk.columns:
            wc_counts = chunk[WEATHER_COL].fillna("Unknown").value_counts()
            for k,v in wc_counts.items():
                weather_counts[k] = weather_counts.get(k, 0) + int(v)

        # road flags
        for col in ROAD_COLS:
            if col in chunk.columns:
                # sum True values (some columns are boolean or 0/1)
                road_counts[col] += int(chunk[col].fillna(False).astype(bool).sum())

        # severity by weather
        if (WEATHER_COL in chunk.columns) and (SEV_COL in chunk.columns):
            pivot = chunk.groupby([WEATHER_COL, SEV_COL]).size()
            for (w,s), count in pivot.items():
                severity_weather.setdefault(w, {}).setdefault(s, 0)
                severity_weather[w][s] += int(count)

        # sample rows for mapping (reservoir sampling up to many points)
        if LAT_COL in chunk.columns and LNG_COL in chunk.columns:
            coords = chunk[[LAT_COL, LNG_COL]].dropna()
            n_take = min(len(coords), 5000)  # take up to 5k per chunk
            if n_take > 0:
                sample_rows.append(coords.sample(n=n_take, random_state=42))

        print(f"Processed chunk {i+1}", end="\r")

    # combine sample rows
    sample_df = pd.concat(sample_rows, ignore_index=True) if sample_rows else pd.DataFrame(columns=[LAT_COL, LNG_COL])
    # convert dicts -> pandas objects
    weather_s = pd.Series(weather_counts).sort_values(ascending=False)
    road_s = pd.Series(road_counts).sort_values(ascending=False)
    # severity by weather -> DataFrame
    sev_weather_df = pd.DataFrame.from_dict(severity_weather, orient="index").fillna(0).sort_values(0, ascending=False)

    return hour_series.astype(int), weekday_series.astype(int), weather_s, road_s, sev_weather_df, sample_df

def group_weather_label(s):
    """Map raw weather_text -> grouped simpler labels"""
    s = str(s).lower()
    if any(x in s for x in ["rain","drizzle","shower","spray"]):
        return "Rain"
    if any(x in s for x in ["snow","sleet","blizzard","flurr", "ice"]):
        return "Snow/Ice"
    if any(x in s for x in ["fog","mist","haze","smoke","squall"]):
        return "Low Visibility"
    if any(x in s for x in ["clear","sun","fair"]):
        return "Clear"
    if any(x in s for x in ["cloud","overcast"]):
        return "Cloudy"
    if any(x in s for x in ["thunder","storm","lightning"]):
        return "Thunderstorm"
    return "Other"

# ---------- MAIN ----------
def main():
    # try full optimized load first
    try:
        df = try_load_with_dtypes(DATA_PATH)
        full_load = True
    except MemoryError:
        df = None
        full_load = False

    if full_load:
        # ensure time column exists and parse
        if TIME_COL not in df.columns:
            raise ValueError(f"{TIME_COL} not found in file.")
        df = df.dropna(subset=[TIME_COL])
        df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
        df = df.dropna(subset=[TIME_COL])

        # time features
        df["hour"] = df[TIME_COL].dt.hour
        df["weekday"] = df[TIME_COL].dt.day_name()
        df["month"] = df[TIME_COL].dt.month_name()

        # group weather to simpler categories
        if WEATHER_COL in df.columns:
            df["weather_group"] = df[WEATHER_COL].apply(group_weather_label)
        else:
            df["weather_group"] = "Unknown"

        # Road booleans - normalize
        present_road_cols = [c for c in ROAD_COLS if c in df.columns]
        for c in present_road_cols:
            df[c] = df[c].fillna(False).astype(bool)

        # --- Aggregations & charts ---
        print("Computing aggregations on full dataset...")
        accidents_by_hour = df["hour"].value_counts().reindex(range(24), fill_value=0).sort_index()
        accidents_by_weekday = df["weekday"].value_counts().reindex(
            ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], fill_value=0
        )
        weather_counts = df["weather_group"].value_counts()
        road_counts = {c: int(df[c].sum()) for c in present_road_cols}

        # severity by weather
        if SEV_COL in df.columns:
            severity_by_weather = df.groupby(["weather_group", SEV_COL]).size().unstack(fill_value=0)
            severity_by_weather.to_csv(os.path.join(OUT_DIR, "severity_by_weather.csv"))

        # save aggregates as CSV
        accidents_by_hour.to_csv(os.path.join(OUT_DIR, "accidents_by_hour.csv"), header=["count"])
        accidents_by_weekday.to_csv(os.path.join(OUT_DIR, "accidents_by_weekday.csv"), header=["count"])
        weather_counts.to_csv(os.path.join(OUT_DIR, "accidents_by_weathergroup.csv"), header=["count"])
        pd.Series(road_counts).to_csv(os.path.join(OUT_DIR, "accidents_by_roadflags.csv"), header=["count"])

        # --- PLOTS ---
        plt.figure(figsize=(10,4))
        plt.plot(accidents_by_hour.index, accidents_by_hour.values, marker="o")
        plt.title("Accidents by Hour")
        plt.xlabel("Hour (0-23)")
        plt.ylabel("Number of Accidents")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "accidents_by_hour.png"), dpi=200)
        plt.close()

        plt.figure(figsize=(9,4))
        accidents_by_weekday.plot(kind="bar")
        plt.title("Accidents by Weekday")
        plt.ylabel("Number of Accidents")
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "accidents_by_weekday.png"), dpi=200)
        plt.close()

        plt.figure(figsize=(9,4))
        weather_counts.head(12).plot(kind="bar")
        plt.title("Top Weather Groups")
        plt.ylabel("Number of Accidents")
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "accidents_by_weather.png"), dpi=200)
        plt.close()

        if road_counts:
            plt.figure(figsize=(10,4))
            pd.Series(road_counts).sort_values(ascending=False).plot(kind="bar")
            plt.title("Accidents near road-condition flags (counts)")
            plt.tight_layout()
            plt.savefig(os.path.join(OUT_DIR, "accidents_by_roadflags.png"), dpi=200)
            plt.close()

        # --- Mapping: sample for HeatMap & clustering ---
        if LAT_COL in df.columns and LNG_COL in df.columns:
            coords = df[[LAT_COL, LNG_COL]].dropna()
            sample_n = min(200_000, len(coords))
            heat_sample = coords.sample(n=sample_n, random_state=42)
            center_lat = float(heat_sample[LAT_COL].median())
            center_lng = float(heat_sample[LNG_COL].median())

            # save heatmap
            m = folium.Map(location=[center_lat, center_lng], zoom_start=6, tiles="CartoDB positron")
            HeatMap(heat_sample.values.tolist(), radius=7, blur=10, min_opacity=0.3).add_to(m)
            m.save(os.path.join(OUT_DIR, "accident_heatmap.html"))

            # DBSCAN clusters on a smaller sample
            cluster_sample = heat_sample.sample(n=min(50_000, len(heat_sample)), random_state=42)
            coords_np = cluster_sample.to_numpy()
            coords_rad = np.radians(coords_np)
            kms_per_radian = 6371.0088

            # ---- UPDATED DBSCAN PARAMETERS ----
            epsilon = 10.0 / kms_per_radian  # 10 km radius
            db = DBSCAN(eps=epsilon, min_samples=30, algorithm="ball_tree", metric="haversine")
            labels = db.fit_predict(coords_rad)

            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            print(f"DBSCAN (full) found {n_clusters} clusters")

            cluster_sample = cluster_sample.reset_index(drop=True).copy()
            cluster_sample["cluster"] = labels

            # create cluster map
            cluster_counts = cluster_sample[cluster_sample.cluster != -1]["cluster"].value_counts().head(20)
            m2 = folium.Map(location=[center_lat, center_lng], zoom_start=6, tiles="CartoDB dark_matter")
            if cluster_counts.empty:
                print("No clusters found in full-load DBSCAN. Try adjusting eps/min_samples.")
            else:
                for cl in cluster_counts.index:
                    member = cluster_sample[cluster_sample.cluster == cl]
                    centroid = [float(member[LAT_COL].mean()), float(member[LNG_COL].mean())]
                    folium.CircleMarker(location=centroid,
                                        radius=8 + math.log(len(member) + 1),
                                        color="red",
                                        fill=True,
                                        fill_opacity=0.7,
                                        popup=f"Cluster {int(cl)} ({len(member)} accidents)").add_to(m2)
            m2.save(os.path.join(OUT_DIR, "top_clusters.html"))

            # save small sample CSV for manual inspection
            heat_sample.sample(n=min(10000, len(heat_sample))).to_csv(os.path.join(OUT_DIR, "map_sample.csv"), index=False)

    else:
        # chunked approach - compute aggregates and sample for mapping
        (accidents_by_hour, accidents_by_weekday,
         weather_counts, road_counts, severity_by_weather,
         sample_df) = load_chunk_aggregations(DATA_PATH)

        # save CSVs
        accidents_by_hour.to_csv(os.path.join(OUT_DIR, "accidents_by_hour.csv"), header=["count"])
        accidents_by_weekday.to_csv(os.path.join(OUT_DIR, "accidents_by_weekday.csv"), header=["count"])
        weather_counts.to_csv(os.path.join(OUT_DIR, "accidents_by_weather_raw.csv"), header=["count"])
        road_counts.to_csv(os.path.join(OUT_DIR, "accidents_by_roadflags_raw.csv"), header=["count"])
        severity_by_weather.to_csv(os.path.join(OUT_DIR, "severity_by_weather_raw.csv"))

        # PLOTS (same appearance as full-load version)
        plt.figure(figsize=(10,4))
        plt.plot(accidents_by_hour.index, accidents_by_hour.values, marker="o")
        plt.title("Accidents by Hour (chunked)")
        plt.xlabel("Hour (0-23)")
        plt.ylabel("Number of Accidents")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "accidents_by_hour_chunked.png"), dpi=200)
        plt.close()

        plt.figure(figsize=(9,4))
        accidents_by_weekday.plot(kind="bar")
        plt.title("Accidents by Weekday (chunked)")
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "accidents_by_weekday_chunked.png"), dpi=200)
        plt.close()

        plt.figure(figsize=(9,4))
        if not weather_counts.empty:
            weather_counts.head(12).plot(kind="bar")
            plt.title("Top Weather Conditions (chunked)")
            plt.tight_layout()
            plt.savefig(os.path.join(OUT_DIR, "accidents_by_weather_chunked.png"), dpi=200)
            plt.close()

        # mapping from sample_df
        if not sample_df.empty:
            sample_df = sample_df.dropna()
            sample_n = min(100_000, len(sample_df))
            heat_sample = sample_df.sample(n=sample_n, random_state=42)
            center_lat = float(heat_sample[LAT_COL].median())
            center_lng = float(heat_sample[LNG_COL].median())

            m = folium.Map(location=[center_lat, center_lng], zoom_start=6, tiles="CartoDB positron")
            HeatMap(heat_sample.values.tolist(), radius=7, blur=10, min_opacity=0.3).add_to(m)
            m.save(os.path.join(OUT_DIR, "accident_heatmap_chunked.html"))

            # DBSCAN on smaller sample
            cluster_sample = heat_sample.sample(n=min(30_000, len(heat_sample)), random_state=42)
            coords_np = cluster_sample.to_numpy()
            coords_rad = np.radians(coords_np)
            kms_per_radian = 6371.0088

            # ---- UPDATED DBSCAN PARAMETERS (chunked) ----
            epsilon = 10.0 / kms_per_radian  # 10 km radius
            db = DBSCAN(eps=epsilon, min_samples=40, algorithm="ball_tree", metric="haversine")
            labels = db.fit_predict(coords_rad)

            n_clusters_chunked = len(set(labels)) - (1 if -1 in labels else 0)
            print(f"DBSCAN (chunked) found {n_clusters_chunked} clusters")

            cluster_sample = cluster_sample.reset_index(drop=True).copy()
            cluster_sample["cluster"] = labels
            cluster_counts = cluster_sample[cluster_sample.cluster != -1]["cluster"].value_counts().head(20)

            m2 = folium.Map(location=[center_lat, center_lng], zoom_start=6, tiles="CartoDB dark_matter")
            if cluster_counts.empty:
                print("No clusters found in chunked DBSCAN. Try adjusting eps/min_samples.")
            else:
                for cl in cluster_counts.index:
                    member = cluster_sample[cluster_sample.cluster == cl]
                    centroid = [float(member[LAT_COL].mean()), float(member[LNG_COL].mean())]
                    folium.CircleMarker(location=centroid,
                                        radius=8 + math.log(len(member) + 1),
                                        color="red",
                                        fill=True,
                                        fill_opacity=0.7,
                                        popup=f"Cluster {int(cl)} ({len(member)} accidents)").add_to(m2)
            m2.save(os.path.join(OUT_DIR, "top_clusters_chunked.html"))

            heat_sample.sample(n=min(5000, len(heat_sample))).to_csv(os.path.join(OUT_DIR, "map_sample_chunked.csv"), index=False)

    print("\n✅ All done. Outputs saved to:", OUT_DIR)
    print("Check these core files in the outputs folder:")
    for f in sorted(os.listdir(OUT_DIR)):
        print(" -", f)

if __name__ == "__main__":
    main()


Attempting optimized full load...
Computing aggregations on full dataset...
DBSCAN (full) found 103 clusters

✅ All done. Outputs saved to: outputs
Check these core files in the outputs folder:
 - accident_heatmap.html
 - accidents_by_hour.csv
 - accidents_by_hour.png
 - accidents_by_roadflags.csv
 - accidents_by_roadflags.png
 - accidents_by_weather.png
 - accidents_by_weathergroup.csv
 - accidents_by_weekday.csv
 - accidents_by_weekday.png
 - hourly_by_weather.csv
 - map_sample.csv
 - sample_inspect.csv
 - severity_by_weather.csv
 - top_clusters.html


In [ ]:
def load_chunk_aggregations(path, chunk_size=250_000):
    """Process CSV in chunks to avoid memory issues and compute summaries."""
    chunker = pd.read_csv(path, parse_dates=[TIME_COL], chunksize=chunk_size, low_memory=True)

    hour_series = pd.Series(0, index=range(24), dtype="int64")
    weekday_series = pd.Series(0, index=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], dtype="int64")
    weather_counts = {}
    road_counts = {col: 0 for col in ROAD_COLS}
    sample_rows = []

    for chunk in chunker:
        chunk = chunk.dropna(subset=[TIME_COL])
        chunk[TIME_COL] = pd.to_datetime(chunk[TIME_COL], errors="coerce")
        chunk["hour"] = chunk[TIME_COL].dt.hour
        chunk["weekday"] = chunk[TIME_COL].dt.day_name()

        # update counters
        hour_series += chunk["hour"].value_counts().reindex(range(24), fill_value=0)
        weekday_series += chunk["weekday"].value_counts().reindex(weekday_series.index, fill_value=0)

        # road flags & weather
        for col in ROAD_COLS:
            if col in chunk.columns:
                road_counts[col] += int(chunk[col].fillna(False).astype(bool).sum())

        if LAT_COL in chunk.columns and LNG_COL in chunk.columns:
            coords = chunk[[LAT_COL, LNG_COL]].dropna()
            if not coords.empty:
                sample_rows.append(coords.sample(n=min(5000, len(coords)), random_state=42))

    sample_df = pd.concat(sample_rows, ignore_index=True) if sample_rows else pd.DataFrame()
    return hour_series, weekday_series, pd.Series(weather_counts), pd.Series(road_counts), sample_df


In [ ]:
# --- Aggregations ---
accidents_by_hour = df["hour"].value_counts().reindex(range(24), fill_value=0).sort_index()
accidents_by_weekday = df["weekday"].value_counts().reindex(
    ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], fill_value=0
)
weather_counts = df["weather_group"].value_counts()

# --- Visualization ---
plt.figure(figsize=(10,4))
plt.plot(accidents_by_hour.index, accidents_by_hour.values, marker="o")
plt.title("Accidents by Hour")
plt.xlabel("Hour (0-23)")
plt.ylabel("Number of Accidents")
plt.grid(True)
plt.savefig(os.path.join(OUT_DIR, "accidents_by_hour.png"), dpi=200)


In [ ]:
coords = df[[LAT_COL, LNG_COL]].dropna().sample(n=200_000, random_state=42)
m = folium.Map(location=[coords[LAT_COL].median(), coords[LNG_COL].median()],
               zoom_start=6, tiles="CartoDB positron")
HeatMap(coords.values.tolist(), radius=7, blur=10, min_opacity=0.3).add_to(m)
m.save(os.path.join(OUT_DIR, "accident_heatmap.html"))

# DBSCAN Clustering
coords_rad = np.radians(coords.to_numpy())
epsilon = 10.0 / 6371.0088  # 10 km
db = DBSCAN(eps=epsilon, min_samples=30, metric="haversine").fit(coords_rad)
labels = db.labels_
print("DBSCAN clusters found:", len(set(labels)) - (1 if -1 in labels else 0))
